# Nanocompore control oligo design 

### Imports

In [3]:
# Standard lib imports
import os
from datetime import date
from collections import *
from glob import glob, iglob
from shutil import rmtree
import itertools
from pprint import pprint as pp

# Generic third party imports
from pycltools.pycltools import *
import pysam
import pyfaidx

# Ploting lib imports
import matplotlib.pyplot as pl
import seaborn as sns
%matplotlib inline

# Data wrangling lib imports
import pandas as pd
import numpy as np
import scipy as sp
pd.options.display.max_colwidth = 200
pd.options.display.max_columns = 200
pd.options.display.max_rows = 50
pd.options.display.min_rows = 50


Bad key "text.kerning_factor" on line 4 in
/nfs/software/birney/adrien/miniconda3/envs/Python3.7/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


# Final design

* m6A: GGACU
* Inosine: UUAGC (loose motif in editing-enriched regions (EERs) – from Blango and Bass 2016, and Eggington et al. 2011).
* PseudoU: UGUAG (from Pus7’s UGΨAR motif, and 7SK IVT peak…)
* m62A: GUGAACC (from the 18S rRNA modified sequence – the two As are modified m62A)
* m5C: CCCGGG (from Huang et al. 2019)
* m1G: CAGGTCG (from the tRNA m1G37 position, e.g. in LeuCAG c1t34 (chromosome 1, tRNA34) isodecoder)
* 2'OmeA: GAGAGAA (from rRNA doi: 10.1093/nar/gkw810)

In [727]:
def get_bases_probs(seq_list):
    base_count = Counter()
    for b in seq_list:
        for i in b.seq:
            base_count[i]+=1
    base_count= pd.Series(base_count)
    p = 1/(base_count/base_count.sum())
    p = p/p.sum()
    return p.index, p.values

def random_gen (length, bases=["A","U","C","G"], seed=49, prob=None):
    np.random.seed(seed)
    l = []
    prev_b = None
    for _ in range(length):
        while True:
            b = np.random.choice(a=bases, p=prob)
            if b == prev_b:
                continue
            else:
                prev_b = b
                l.append(b)
                break
    return "".join(l)

## Brute force generate sequence

In [728]:
block = namedtuple("block", ["name", "len", "seq", "seq_mod"])

blocks_1 = [
    block("m6A_strong",5, "GGACU", "GG(m6A)CU"),
    block("m6A_weak",5, "CGACC", "CG(m6A)CC"),
    block("m6A_anti",5, "CUAGC", "CU(m6A)GC")]
    
blocks_2 = [
    block("Inosine",5, "UUAGC", "UU(I)GC"),
    block("PseudoU",5, "UGUAG", "UG(~U)AG"),
    block("m5C",4, "CCGG", "C(5-M-C)GG")]

blocks_3 = [
    block("m1G",7, "CAGGUCG", "CAG(m1G)UCG"),
    block("m62A",7, "GUGAACC", "GUG(DMA)ACC"),
    block("2OmeA",5, "GAGAGAA", "GAG(mA)GAA")]

#bases, prob = get_bases_probs (seq_list = blocks_1+blocks_2+blocks_3)
#print (bases, prob)

init_buffer_len = 16
block_size = 7
final_buffer_len = 9
polyA_len=10
random_rounds = 1000

# Generate all possible block ordering
b1_perm = itertools.permutations(blocks_1, len(blocks_1))
b2_perm = itertools.permutations(blocks_2, len(blocks_2))
b3_perm = itertools.permutations(blocks_3, len(blocks_3))

n = 0
mkdir("results")
fa_fn = "results/permuted_seq.fa"

with open (fa_fn, "w") as fa_fp: 
    for blocks_shuffled_1, blocks_shuffled_2, blocks_shuffled_3 in tqdm(itertools.product(b1_perm, b2_perm, b3_perm)):
        for seed in range(random_rounds):
            s = seed+1
            seq_list = []
            name_list = []
            seq_list.append(randon_gen(length=init_buffer_len, seed=seed, bases=["A","U","C","G"]))
            s+=1

            for blocks in zip(blocks_shuffled_1, blocks_shuffled_2, blocks_shuffled_3):
                for b in blocks:
                    seq_list.append(b.seq)
                    seq_list.append(randon_gen(length=block_size-b.len, seed=seed, bases=["A","U","C","G"]))
                    name_list.append(b.name)
                    s+=1

            seq_list.append(randon_gen(length=final_buffer_len, seed=seed, bases=["A","U","C","G"]))
            seq_list.append("A"*polyA_len)

            name =  "-".join(name_list)+f"|seed={seed}"
            seq = "".join(seq_list)
            fa_fp.write(f">{name}\n{seq}\n")
            n+=1

print (f"{n} sequences generated")
head(fa_fn, max_char_line=200, max_char_col=200)

216it [02:10,  1.65it/s]

216000 sequences generated
>m6A_strong-Inosine-m1G-m6A_weak-PseudoU-m62A-m6A_anti-m5C-2OmeA|seed=0                              
AGUAGUGUCAGCACUCGGACUAGUUAGCAGCAGGUCGCGACCAGUGUAGAGGUGAACCCUAGCAGCCGGAGUGAGAGAAAGAGUAGUGUCAAAAAAAAAA 
>m6A_strong-Inosine-m1G-m6A_weak-PseudoU-m62A-m6A_anti-m5C-2OmeA|seed=1                              
UGAGUGUGAUAGUACUGGACUUGUUAGCUGCAGGUCGCGACCUGUGUAGUGGUGAACCCUAGCUGCCGGUGAGAGAGAAUGUGAGUGUGAAAAAAAAAAA 
>m6A_strong-Inosine-m1G-m6A_weak-PseudoU-m62A-m6A_anti-m5C-2OmeA|seed=2                              
AGUACGCGAGCUGUGCGGACUAGUUAGCAGCAGGUCGCGACCAGUGUAGAGGUGAACCCUAGCAGCCGGAGUGAGAGAAAGAGUACGCGAAAAAAAAAAA 
>m6A_strong-Inosine-m1G-m6A_weak-PseudoU-m62A-m6A_anti-m5C-2OmeA|seed=3                              
CAUGAUGCGUCUGCAUGGACUCAUUAGCCACAGGUCGCGACCCAUGUAGCAGUGAACCCUAGCCACCGGCAUGAGAGAACACAUGAUGCGAAAAAAAAAA 
>m6A_strong-Inosine-m1G-m6A_weak-PseudoU-m62A-m6A_anti-m5C-2OmeA|seed=4                              
CGUAGACUCAGUCGUAGGACUCGUUAGCCGCAGGUCGCGACCCGUGUAGCGGUGA

## Fold with RNAFold

In [4]:
bash("RNAfold --version", conda="ViennaRNA")

RNAfold 2.4.15


In [729]:
threads = 25
mem = 20000

fa_fn = "results/permuted_seq.fa"
res_fn = "results/rnafold_res.tsv"
stdout_fp = "results/rna_fold.out"
stderr_fp = "results/rna_fold.err"

cmd = f"RNAfold --noPS -i {fa_fn} --jobs={threads} > {res_fn}"
bsub (conda="ViennaRNA", cmd=cmd, threads=threads, mem=mem, stdout_fp=stdout_fp, stderr_fp=stderr_fp)

bsub -M 20000 -R 'rusage[mem=20000]' -n 25 -oo results/rna_fold.out -eo results/rna_fold.err  "RNAfold --noPS -i results/permuted_seq.fa --jobs=25 > results/rnafold_res.tsv"
Job <3330156> is submitted to default queue <research-rh74>.


'3330156'

In [732]:
res_fn = "./results/rnafold_res.tsv"

seq_list = []
seq_info = namedtuple ("seq_info", ["name", "folding_score",  "base_distrib", "base_balance", "cg_percent", "seq", "folding_string"])


with open (res_fn) as fp:
    score = -1
    seq = ""
    while True:
        try:
            header, seq, fold = next(fp), next(fp), next(fp)
        except StopIteration:
            break
            
        # extract values from RNA fold file
        name = header[1:].strip()
        seq = seq.strip()
        folding_string = fold.strip().split()[0]
        folding_score = -float(fold.rpartition(")")[0].rpartition("(")[2].strip())
        
        # Compute base content
        c = Counter()
        for base in (seq[:-10]):
            c[base]+=1
        base_balance = np.std(list(c.values()))
        cg_percent = (c["C"]+c["G"])/np.sum(list(c.values()))
        
        # Save values values
        seq_list.append (seq_info (name, folding_score, c, base_balance, cg_percent, seq, folding_string))
        
df = pd.DataFrame(seq_list)
df["combined_score"] = df["folding_score"]+df["base_balance"]
df.sort_values(by="combined_score", inplace=True)
df.head(100)

,name,folding_score,base_distrib,base_balance,cg_percent,seq,folding_string,combined_score
211340,m6A_anti-m5C-m1G-m6A_weak-PseudoU-2OmeA-m6A_strong-Inosine-m62A|seed=340,11.7,"{'A': 25, 'U': 22, 'C': 21, 'G': 22}",1.500000,0.477778,AUCUGUAGCUCGACACCUAGCAUCCGGAUCCAGGUCGCGACCAUUGUAGAUGAGAGAAAUGGACUAUUUAGCAUGUGAACCAUCUGUAGCAAAAAAAAAA,.(((((..(((..((.(((((..((.......))..))........))).)).)))..))))).......(((.(((...))).))).............,13.200000
44802,m6A_strong-Inosine-m62A-m6A_anti-m5C-m1G-m6A_weak-PseudoU-2OmeA|seed=802,9.5,"{'A': 30, 'U': 21, 'C': 19, 'G': 20}",4.387482,0.433333,AUACUCGACAUAGAUAGGACUAUUUAGCAUGUGAACCCUAGCAUCCGGAUACAGGUCGCGACCAUUGUAGAUGAGAGAAAUAUACUCGACAAAAAAAAAA,....((.(((((((((....))))))...)))))...((..((((....(((((((....)))..))))))))..)).......................,13.887482
178802,m6A_anti-m5C-2OmeA-m6A_strong-PseudoU-m1G-m6A_weak-Inosine-m62A|seed=802,9.5,"{'A': 30, 'U': 21, 'C': 19, 'G': 20}",4.387482,0.433333,AUACUCGACAUAGAUACUAGCAUCCGGAUAGAGAGAAAUGGACUAUUGUAGAUCAGGUCGCGACCAUUUAGCAUGUGAACCAUACUCGACAAAAAAAAAA,....((((....(((.(((.((((((............))))....)))))))).((((((.............))).)))....))))...........,13.887482
213639,m6A_anti-m5C-m62A-m6A_weak-PseudoU-2OmeA-m6A_strong-Inosine-m1G|seed=639,10.4,"{'A': 26, 'C': 26, 'U': 18, 'G': 20}",3.570714,0.511111,ACUCUGUAUCAUACGACUAGCACCCGGACUGUGAACCCGACCACUGUAGACGAGAGAAACGGACUACUUAGCACCAGGUCGACUCUGUAUAAAAAAAAAA,....(((((....(((((......(((.........)))......((((.((.......))..)))).........))))).....))))).........,13.970714
207639,m6A_anti-m5C-m62A-m6A_weak-Inosine-2OmeA-m6A_strong-PseudoU-m1G|seed=639,10.4,"{'A': 26, 'C': 26, 'U': 18, 'G': 20}",3.570714,0.511111,ACUCUGUAUCAUACGACUAGCACCCGGACUGUGAACCCGACCACUUAGCACGAGAGAAACGGACUACUGUAGACCAGGUCGACUCUGUAUAAAAAAAAAA,....(((((....(((((......((..(((((........)))..))..))......((((....))))......))))).....))))).........,13.970714
76639,m6A_weak-Inosine-2OmeA-m6A_strong-PseudoU-m1G-m6A_anti-m5C-m62A|seed=639,10.8,"{'A': 26, 'C': 26, 'U': 18, 'G': 20}",3.570714,0.511111,ACUCUGUAUCAUACGACGACCACUUAGCACGAGAGAAACGGACUACUGUAGACCAGGUCGCUAGCACCCGGACUGUGAACCACUCUGUAUAAAAAAAAAA,....((..(((((...((.....(((((((.......((((....)))).......)).)))))....))...)))))..))..................,14.370714
193639,m6A_anti-PseudoU-m1G-m6A_weak-Inosine-2OmeA-m6A_strong-m5C-m62A|seed=639,11.0,"{'A': 26, 'C': 26, 'U': 18, 'G': 20}",3.570714,0.511111,ACUCUGUAUCAUACGACUAGCACUGUAGACCAGGUCGCGACCACUUAGCACGAGAGAAACGGACUACCCGGACUGUGAACCACUCUGUAUAAAAAAAAAA,.........................((((...(((((((.((...(((..((.......))..)))...))..)))).)))..)))).............,14.570714
12802,m6A_strong-PseudoU-m1G-m6A_weak-Inosine-m62A-m6A_anti-m5C-2OmeA|seed=802,10.2,"{'A': 30, 'U': 21, 'C': 19, 'G': 20}",4.387482,0.433333,AUACUCGACAUAGAUAGGACUAUUGUAGAUCAGGUCGCGACCAUUUAGCAUGUGAACCCUAGCAUCCGGAUAGAGAGAAAUAUACUCGACAAAAAAAAAA,....((((.(((.......(((((...(((((((((((.............))))..))).).)))..))))).......)))..))))...........,14.587482
211802,m6A_anti-m5C-m1G-m6A_weak-PseudoU-2OmeA-m6A_strong-Inosine-m62A|seed=802,10.2,"{'A': 30, 'U': 21, 'C': 19, 'G': 20}",4.387482,0.433333,AUACUCGACAUAGAUACUAGCAUCCGGAUACAGGUCGCGACCAUUGUAGAUGAGAGAAAUGGACUAUUUAGCAUGUGAACCAUACUCGACAAAAAAAAAA,....((((........((..((((....(((((((....)))..))))))))..))..((((..(((.......)))..))))..))))...........,14.587482
12203,m6A_strong-PseudoU-m1G-m6A_weak-Inosine-m62A-m6A_anti-m5C-2OmeA|seed=203,9.9,"{'A': 30, 'U': 23, 'C': 18, 'G': 19}",4.716991,0.411111,AUACAUAUCAGUACGUGGACUAUUGUAGAUCAGGUCGCGACCAUUUAGCAUGUGAACCCUAGCAUCCGGAUAGAGAGAAAUAUACAUAUCAAAAAAAAAA,.......((..((..((((....(((((.(((.((.((.........)))).)))...)).)))))))..))..))........................,14.616991


## Extract best candidate

In [736]:
sequence_ID = 44802

block = namedtuple("block", ["len", "seq", "seq_mod", "set"])

block_dict = {
    "m6A_strong": block(5, "GGACU", "GG(m6A)CU", "mod_set_1"),
    "m6A_weak": block(5, "CGACC", "CG(m6A)CC", "mod_set_1", ),
    "m6A_anti": block(5, "CUAGC", "CU(m6A)GC", "mod_set_1"),
    "Inosine": block(5, "UUAGC", "UU(I)GC", "mod_set_2"),
    "PseudoU": block(5, "UGUAG", "UG(~U)AG", "mod_set_2", ),
    "m5C": block(4, "CCGG", "C(5-M-C)GG", "mod_set_2"),
    "m1G": block(7, "CAGGUCG", "CAG(m1G)UCG", "mod_set_3"),
    "m62A": block(7, "GUGAACC", "GUG(DMA)ACC", "mod_set_3", ),
    "2OmeA": block(5, "GAGAGAA", "GAG(mA)GAA","mod_set_3")}

init_buffer_len = 16
block_size = 7
final_buffer_len = 9
polyA_len=10

candidate = df.loc[sequence_ID]
seq = candidate["seq"]
name = candidate["name"]
print(name)

block_order = name.split("|")[0].split("-")
seed = int( name.split("=")[-1])

seq_dict = defaultdict(list)

init_buffer = randon_gen(length=init_buffer_len, seed=seed, bases=["A","U","C","G"])
for b_type in ("control", "mod_set_1", "mod_set_2", "mod_set_3"):
    seq_dict[b_type].append(init_buffer)
seed+=1

for b_name in block_order:
    b = block_dict[b_name]
    spacer = randon_gen(length=block_size-b.len, seed=seed, bases=["A","U","C","G"])
    for b_type in ("control", "mod_set_1", "mod_set_2", "mod_set_3"):
        if b.set == b_type:
            b_seq = b.seq_mod
        else:
            b_seq = b.seq
            
        seq_dict[b_type].append(b_seq)
        seq_dict[b_type].append(spacer)
    seed+=1

final_buffer = randon_gen(length=final_buffer_len, seed=seed, bases=["A","U","C","G"])
for b_type in ("control", "mod_set_1", "mod_set_2", "mod_set_3"):
    seq_dict[b_type].append(final_buffer)
    seq_dict[b_type].append("A"*polyA_len)

for i, j in seq_dict.items():
    print (f">{i}")
    print("".join(j))
    print (seq)

m6A_strong-Inosine-m62A-m6A_anti-m5C-m1G-m6A_weak-PseudoU-2OmeA|seed=802
>control
AUACUCGACAUAGAUAGGACUCUUUAGCUAGUGAACCCUAGCCUCCGGAGACAGGUCGCGACCUGUGUAGAUGAGAGAACUGAGUGCACAAAAAAAAAAA
>mod_set_1
AUACUCGACAUAGAUAGG(m6A)CUCUUUAGCUAGUGAACCCU(m6A)GCCUCCGGAGACAGGUCGCG(m6A)CCUGUGUAGAUGAGAGAACUGAGUGCACAAAAAAAAAAA
>mod_set_2
AUACUCGACAUAGAUAGGACUCUUU(I)GCUAGUGAACCCUAGCCUC(5-M-C)GGAGACAGGUCGCGACCUGUG(~U)AGAUGAGAGAACUGAGUGCACAAAAAAAAAAA
>mod_set_3
AUACUCGACAUAGAUAGGACUCUUUAGCUAGUG(DMA)ACCCUAGCCUCCGGAGACAG(m1G)UCGCGACCUGUGUAGAUGAG(mA)GAACUGAGUGCACAAAAAAAAAAA
